In [7]:
pip install pyspark

In [93]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, StringType, DateType
import random
from datetime import datetime, timedelta

In [103]:
def get_name():  #Получение рандомного имени из списка

    names = [
    'James', 'Mary', 'John', 'Patricia', 'Robert', 'Jennifer', 'Michael', 'Linda',
    'William', 'Elizabeth', 'David', 'Barbara', 'Richard', 'Susan', 'Joseph', 'Jessica',
    'Charles', 'Sarah', 'Thomas', 'Karen', 'Daniel', 'Nancy', 'Matthew', 'Lisa',
    'Anthony', 'Betty', 'Mark', 'Dorothy', 'Paul', 'Helen', 'Steven', 'Sandra',
    'Andrew', 'Ashley', 'Joshua', 'Kimberly', 'Kenneth', 'Donna', 'Kevin', 'Carol',
    'Brian', 'Michelle', 'George', 'Amanda', 'Edward', 'Melissa', 'Ronald', 'Deborah',
    'Timothy', 'Stephanie', 'Jason', 'Rebecca', 'Jeffrey', 'Laura', 'Ryan', 'Sharon',
    'Jacob', 'Cynthia', 'Gary', 'Kathleen', 'Nicholas', 'Angela', 'Eric', 'Helen',
    'Jonathan', 'Janet', 'Justin', 'Ruth', 'Jordan', 'Maria', 'Austin', 'Katherine',
    'Ethan', 'Christine', 'Evelyn', 'Alice', 'Terry', 'Megan', 'Sean', 'Carolyn',
    'Alex', 'Diana', 'Benjamin', 'Kathy', 'Adam', 'Margaret', 'Patrick', 'Emily',
    'Larry', 'Joan', 'Jerry', 'Teresa', 'Tyler', 'Tammy', 'Aaron', 'Anna'
]
    return random.choice(names)

In [95]:
def get_cities(): # Получение рандомного города из списка
    cities = ["Paris", "London", "Berlin", "Madrid", "Rome", "Oslo", "Minsk",
    "Lima", "Tokyo", "Houston", "Malmo", "Berlin", "Denver", "Stockholm",
    "Dublin", "Vienna", "Kiev", "Brussels", "Lisbon", "Seoul", "Moscow",
    "Prague", "Warsaw", "Budapest", "Florence", "Munich", "Naples", "SanDiego",
    "Sofia", "Athens", "Zurich", "Dubai", "Cairo", "Helsinki", "Krakow",
    "Chennai", "Lagos", "Kabul", "Riga", "Nairobi", "Doha", "Brno",
    "Osaka", "Hanoi", "Dhaka", "Baku", "Tbilisi", "Bali", "Kabul",
    "Quito", "Antwerp", "Leeds", "Fargo", "Kiev", "Santo", "Ibiza",
    "Sanaa", "Qingdao", "Lyon", "Syracuse", "Catania", "Taipei", "Manila",
    "Catania", "Laredo", "Essen", "Ghent", "Bogota", "Catania", "Casablanca"
]
    return random.choice(cities)

In [96]:
def get_email(name): #Получаем емейл: приводим имя к нижнему регистру, добавляем рандомный инт, добавляем рандомный домен из списка
    domains = ['mail.ru', 'gmail.com', 'inbox.ru', 'outlook.com', 'yahoo.com']
    return f"{name.lower()}.{random.randint(1, 10000)}@{random.choice(domains)}"

In [97]:
def generate_age(): #Генерируем возраст - инт
  return random.randint(18, 50)

In [98]:
def get_salary(): #Генерируем зарплату с шагом 10к
  r_salary = list(range(100000, 500001, 10000))
  return random.choice(r_salary)

In [99]:
def get_registration_date(age): #Генерируем дату, умножаем возраст на количсетво дней, приводим к нужному формату
  base_date = datetime.now() - timedelta(days=365 * age)
  return base_date.strftime('%Y-%m-%d')

In [100]:
#Делаем удфки, чтобы спарк видел наши функции
udf_name = udf(generate_name, StringType())
udf_age = udf(generate_age, IntegerType())
udf_salary = udf(get_salary, IntegerType())
udf_city = udf(get_cities, StringType())
udf_email = udf(lambda name: get_email(name), StringType())
udf_date = udf(lambda age: get_registration_date(age), StringType())

In [101]:
def create_data_frame(num_rows=int):  #финальная функция которая создает таблицу с использованием pySpark
  spark = SparkSession.builder.appName('GenerateData').getOrCreate()  #конфиг не создаю, не вижу смысла

  df = spark.range(num_rows) \
        .withColumn('id', col('id').cast('integer')) \
        .withColumn('name', udf_name()) \
        .withColumn('age', udf_age()) \
        .withColumn('salary', udf_salary()) \
        .withColumn('city', udf_city()) \
        .withColumn('email', udf_email(col('name'))) \
        .withColumn('registration_date', udf_date(col('age')))

  return df

In [87]:
num_rows = int(input('Введите число строк для генерации данных: '))

Введите число строк для генерации данных: 1000


In [102]:
df = create_data_frame(num_rows)  #Запускаем функцию
output_file = datetime.now().strftime('%Y-%m-%d') + '-dev.csv'
df.coalesce(1).write.mode('overwrite').csv(output_file, header=True) #Сохраняем файл, используем коляску, чтобы файл не разделился на несколько блоков
